# 101: PyTorch classification

From Machine Learning model to scalable deployment with Analitico.

Read the full documentation in the [Documents](https://analitico.ai/app/recipes/rx_pytorch_classification/markdown/readme.md) page.

In [3]:
# Load some dependencies
!pip install -q torch

## Model
We define a simple neural network with PyTorch.

In [5]:
import logging
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

import analitico

import torch
import torch.nn as nn
from torch.autograd import Variable

import numpy as np
import pandas as pd

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# You can import your own library.
# Here we import our custom model.
# See: https://analitico.ai/app/recipes/rx_pytorch_classification/files/Net.py
from Net import Net

# Use CPU or GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load sample dataset
iris = datasets.load_iris()

X = iris.data.astype('float32')
y = iris.target.astype('int')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

# Create model, optimizer and loss function
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()
epochs = 100

# Train the model without dataloader
X_train_var, y_train_var = Variable(torch.from_numpy(X_train)).float(), Variable(torch.from_numpy(y_train)).long()

min_loss = 2.0
for epoch in range(epochs):
    y_pred = model(X_train_var)
    loss = loss_fn(y_pred, y_train_var)

    # Zero gradients
    optimizer.zero_grad()
    # Gradients
    loss.backward()
    # Update
    optimizer.step()
    
    # Save the best model
    if loss.item() < min_loss:
        min_loss = loss.item()
        torch.save(model, 'pytorch_classifer_model.pt')

print("Training loss: ", min_loss)

# Test the model accuracy
X_test_var = Variable(torch.from_numpy(X_test)).float()
pred = model(X_test_var)
pred = pred.detach().numpy()

score = accuracy_score(y_test, np.argmax(pred, axis=1))
print ("Model accuracy: {:.2%}".format(score))

# Save metrics for later training evaluation
analitico.set_metric("loss", str(min_loss), "Loss", "best-model", "Best model")
analitico.set_metric("accuracy", str(score), "Accuracy", "best-model")

Training loss:  0.570770263671875
Model accuracy: 96.67%


## Analitico serverless handler

The serverless endpoint is set to call the `handle(event, **kwargs)` method with a set of data required by the model.
Here we load the model, execute the prediction with the input data and return the result.

In [6]:
# This cell is executed in a different context.
# All imports are declared here and all the dependecies 
# it needs to run are specified in the requirements.txt file
# https://analitico.ai/app/recipes/rx_pytorch_classification/files/requirements.txt.

import json
import numpy as np
import torch
from torch.autograd import Variable

# Import our custom model.
# See: https://analitico.ai/app/recipes/rx_pytorch_classification/files/Net.py
from Net import Net

# Retrieve the model here to be used between requests
model = torch.load("pytorch_classifer_model.pt")
model.eval()

def handle(event, **kwargs):
    """ Method called by the serverless endpoint url with the data for prediction """
    
    event = np.array([event])

    # Convert to NDArray
    data = Variable(torch.from_numpy(event)).float()
    
    prediction = model(data).detach().numpy()
    
    class_id = np.argmax(prediction)
    
    return { "class_id": int(class_id) }

def test(**kwargs):
    """ Test method can be called by Analitico to verify that model is working """
    
    # sepal_length, sepal_width, petal_length, petal_width
    data = [6.3, 2.3, 4.4, 1.3]
        
    results = handle(data)
    print(json.dumps(results, indent=2))
  
    # we expect our prediction to be the iris Versicolour (class 1)
    print("\nTest passed: ")
    return results["class_id"] == 1

## Test the endpoint

Finally, let's test the endpoint with sample data.

In [7]:
test()

{
  "class_id": 1
}

Test passed: 


True